In [9]:
import numpy as np
import matplotlib.pyplot as plt


mhatree_to_kcal = 0.6275094740631

train_set, eval__set = [], []
train_set.append([185.89158003098848, 445.34157216716665])
eval__set.append([178.75854150564686, 662.4498123299053])

train_set.append([68.57623712544817, 149.1687403205493])
eval__set.append([108.49399374936273, 669.175])

# train_set.append([146.696, 376.323, 1159])
# eval__set.append([169.518, 632.872, 244])

train_set, eval__set = np.array(train_set), np.array(eval__set)
train_set = mhatree_to_kcal * train_set

print(np.array_str(train_set[-1, :], precision=2, suppress_small=True))
print(np.array_str(eval__set[-1, :], precision=2, suppress_small=True))

[43.03 93.6 ]
[108.49 669.17]


In [ ]:
plt.plot(train_set[:, 0], "r")
plt.plot(eval__set[:, 0], "b")
plt.plot(train_set[:, 1], "r--")
plt.plot(eval__set[:, 1], "b--")
plt.show()

In [1]:
import numpy as np
print(np.linspace(-0.25, 0.25, 11))
print(np.linspace(-0.225, 0.225, 10))
print(np.linspace(-0.5, -0.275, 10))
print(np.linspace(0.275, 0.5, 10))

print(np.linspace(-0.5, 0.5, 21))


[-0.25 -0.2  -0.15 -0.1  -0.05  0.    0.05  0.1   0.15  0.2   0.25]
[-0.225 -0.175 -0.125 -0.075 -0.025  0.025  0.075  0.125  0.175  0.225]
[-0.5   -0.475 -0.45  -0.425 -0.4   -0.375 -0.35  -0.325 -0.3   -0.275]
[0.275 0.3   0.325 0.35  0.375 0.4   0.425 0.45  0.475 0.5  ]
[-0.5  -0.45 -0.4  -0.35 -0.3  -0.25 -0.2  -0.15 -0.1  -0.05  0.    0.05
  0.1   0.15  0.2   0.25  0.3   0.35  0.4   0.45  0.5 ]


python cadft/train_model.py -dl -0.25 0.25 11 -b cc-pvdz --extend_atom 0 1 --extend_xyz 0 1 2 --eval_step 25 --load checkpoint2024-04-30-16-02-30;python gen_dm_ene.py -dl -0.5 -0.275 5 -b cc-pvdz --extend_atom 0 1 --extend_xyz 0 1 2 --name_mol Methane Ethane Ethylene Acetylene Allene Cyclopropene Propyne Cyclopropane Propylene Propane Isobutane Butane Butadiene Butyne Bicyclobutane Cyclopropylmethyl Cyclobutane Spiropentane Benzene Pentane Isopentane Neopentane Cyclopentane;python gen_dm_ene.py -dl 0.275 0.5 10 -b cc-pvdz --extend_atom 0 1 --extend_xyz 0 1 2 --name_mol Methane Ethane Ethylene Acetylene Allene Cyclopropene Propyne Cyclopropane Propylene Propane Isobutane Butane Butadiene Butyne Bicyclobutane Cyclopropylmethyl Cyclobutane Spiropentane Benzene Pentane Isopentane Neopentane Cyclopentane;python gen_dm_ene.py -dl -0.225 0.225 10 -b cc-pvdz --extend_atom 0 1 --extend_xyz 0 1 2 --name_mol Methane Ethane Ethylene Acetylene Allene Cyclopropene Propyne Cyclopropane Propylene Propane Isobutane Butane Butadiene Butyne Bicyclobutane Cyclopropylmethyl Cyclobutane Spiropentane Benzene Pentane Isopentane Neopentane Cyclopentane